In [1]:
import os, json, time
import pandas as pd
import joblib

from datetime import datetime
import mlflow
import mlflow.sklearn
from xgboost import XGBClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# =========================
# Config
# =========================
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("CooperativeCreditRisk-XGBoost")

os.makedirs("../artifacts/models", exist_ok=True)
os.makedirs("../artifacts/reports", exist_ok=True)

FEATURES = [
    "duration","credit_amount","age",
    "checking_status","employment","savings_status","purpose"
]
TARGET = "target"

fixed_test_path   = "../artifacts/dataset/fixed_test.csv"
base50_path       = "../artifacts/dataset/initial_base_50.csv"
future_pool_path  = "../artifacts/dataset/future_pool.csv"
feedback_path     = "../artifacts/dataset/feedback_new_data.csv"  # igual que app
registry_path     = "../artifacts/models/model_registry.json"

batch_size = 10

def log(msg):
    ts = datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] {msg}", flush=True)

def ensure_cols(df, cols, name):
    miss = [c for c in cols if c not in df.columns]
    if miss:
        raise ValueError(f"{name}: faltan columnas {miss}")

def evaluate(model, X_eval, y_eval):
    proba = model.predict_proba(X_eval)[:, 1]
    pred = (proba >= 0.5).astype(int)
    return {
        "accuracy": float(accuracy_score(y_eval, pred)),
        "f1": float(f1_score(y_eval, pred)),
        "precision": float(precision_score(y_eval, pred)),
        "recall": float(recall_score(y_eval, pred)),
        "roc_auc": float(roc_auc_score(y_eval, proba)),
    }

# =========================
# Load fixed test
# =========================
if not os.path.exists(fixed_test_path):
    raise FileNotFoundError(f"fixed_test no encontrado: {fixed_test_path}")

fixed_test = pd.read_csv(fixed_test_path)
ensure_cols(fixed_test, FEATURES + [TARGET], "fixed_test")
fixed_test[TARGET] = fixed_test[TARGET].astype(int)

X_test = fixed_test[FEATURES].copy()
y_test = fixed_test[TARGET].copy()

# =========================
# Load base50
# =========================
base50 = pd.read_csv(base50_path)
ensure_cols(base50, FEATURES + [TARGET], "base50")
base50[TARGET] = base50[TARGET].astype(int)

# =========================
# Load future_pool + take batch
# =========================
future_pool = pd.read_csv(future_pool_path)
ensure_cols(future_pool, FEATURES + [TARGET], "future_pool")
future_pool[TARGET] = future_pool[TARGET].astype(int)

take_n = min(batch_size, len(future_pool))
if take_n == 0:
    batch = future_pool.copy()
    future_remaining = future_pool.copy()
else:
    batch = future_pool.sample(take_n, random_state=7)
    future_remaining = future_pool.drop(batch.index)

# persist remaining
future_remaining.to_csv(future_pool_path, index=False)

# =========================
# Load feedback (from app)
# =========================
if os.path.exists(feedback_path):
    feedback = pd.read_csv(feedback_path)
    ensure_cols(feedback, FEATURES + [TARGET], "feedback")
    feedback[TARGET] = feedback[TARGET].astype(int)
else:
    feedback = pd.DataFrame(columns=FEATURES + [TARGET])

log(f"Base50 rows: {len(base50)}")
log(f"Future pool before: {len(future_pool)}")
log(f"Batch taken: {len(batch)}")
log(f"Future pool after: {len(future_remaining)}")
log(f"Feedback rows: {len(feedback)}")

# =========================
# Train set: base50 + batch + feedback
# =========================
train_plus = pd.concat([base50, batch, feedback], ignore_index=True)
ensure_cols(train_plus, FEATURES + [TARGET], "train_plus")

X_train = train_plus[FEATURES].copy()
y_train = train_plus[TARGET].astype(int).copy()

log(f"Train total rows: {len(train_plus)}")
log(f"Fixed test rows: {len(X_test)}")

# =========================
# Pipeline
# =========================
cat_cols = ["checking_status","employment","savings_status","purpose"]
num_cols = ["duration","credit_amount","age"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ],
    remainder="drop"
)

xgb = XGBClassifier(
    n_estimators=450,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.90,
    colsample_bytree=0.90,
    random_state=42,
    n_jobs=-1,
    eval_metric="logloss",
    verbosity=1
)

pipe = Pipeline([("preprocess", preprocess), ("model", xgb)])

# version name
version = "v_" + datetime.utcnow().strftime("%Y%m%d_%H%M%S")

# =========================
# Train with visible progress
# =========================
with mlflow.start_run(run_name=f"retrain_{version}") as run:
    t0 = time.time()

    log("Training start (with XGBoost verbose eval)...")

    # transform first to pass eval_set to XGBClassifier
    pre = pipe.named_steps["preprocess"]
    model = pipe.named_steps["model"]

    Xtr = pre.fit_transform(X_train)
    Xev = pre.transform(X_test)

    model.fit(
        Xtr, y_train,
        eval_set=[(Xev, y_test)],
        verbose=True
    )

    log(f"Training finished in {time.time() - t0:.2f}s")

    metrics = evaluate(pipe, X_test, y_test)

    mlflow.log_param("deployment_version", version)
    mlflow.log_param("batch_size_used", int(len(batch)))
    mlflow.log_param("future_pool_remaining", int(len(future_remaining)))
    mlflow.log_param("base_rows", int(len(base50)))
    mlflow.log_param("feedback_rows", int(len(feedback)))
    mlflow.log_param("train_total_rows", int(len(train_plus)))
    mlflow.log_param("test_rows_fixed", int(len(X_test)))

    for k, v in metrics.items():
        mlflow.log_metric(k, v)

    model_path = f"../artifacts/models/model_{version}.joblib"
    joblib.dump(pipe, model_path)

    metrics_path = f"../artifacts/reports/metrics_{version}.json"
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)

    mlflow.log_artifact(model_path, artifact_path="model_joblib")
    mlflow.log_artifact(metrics_path, artifact_path="reports")
    mlflow.sklearn.log_model(
    pipe,
    artifact_path="sklearn_model",
    registered_model_name="CooperativeCreditRisk-XGBoost"
)


log("New version: " + version)
log("Metrics: " + str(metrics))

# =========================
# Update registry
# =========================
with open(registry_path, "r") as f:
    registry = json.load(f)

registry["current_version"] = version
registry["models"][version] = {
    "path": f"artifacts/models/model_{version}.joblib",
    "metrics_path": f"artifacts/reports/metrics_{version}.json"
}

with open(registry_path, "w") as f:
    json.dump(registry, f, indent=2)

log("Updated registry: " + registry_path)


[15:30:29] Base50 rows: 50
[15:30:29] Future pool before: 340
[15:30:29] Batch taken: 10
[15:30:29] Future pool after: 330
[15:30:29] Feedback rows: 7
[15:30:29] Train total rows: 67
[15:30:29] Fixed test rows: 250
[15:30:29] Training start (with XGBoost verbose eval)...
[0]	validation_0-logloss:0.60796
[1]	validation_0-logloss:0.60365
[2]	validation_0-logloss:0.60065
[3]	validation_0-logloss:0.59878
[4]	validation_0-logloss:0.59757
[5]	validation_0-logloss:0.59668
[6]	validation_0-logloss:0.59672
[7]	validation_0-logloss:0.59508
[8]	validation_0-logloss:0.59294
[9]	validation_0-logloss:0.59172
[10]	validation_0-logloss:0.59176
[11]	validation_0-logloss:0.59091
[12]	validation_0-logloss:0.59158
[13]	validation_0-logloss:0.59036
[14]	validation_0-logloss:0.59020
[15]	validation_0-logloss:0.59109
[16]	validation_0-logloss:0.59046
[17]	validation_0-logloss:0.59116
[18]	validation_0-logloss:0.59203
[19]	validation_0-logloss:0.59293
[20]	validation_0-logloss:0.59325
[21]	validation_0-loglos

[230]	validation_0-logloss:0.74605
[231]	validation_0-logloss:0.74593
[232]	validation_0-logloss:0.74731
[233]	validation_0-logloss:0.74818
[234]	validation_0-logloss:0.74890
[235]	validation_0-logloss:0.74932
[236]	validation_0-logloss:0.74990
[237]	validation_0-logloss:0.75185
[238]	validation_0-logloss:0.75257
[239]	validation_0-logloss:0.75251
[240]	validation_0-logloss:0.75307
[241]	validation_0-logloss:0.75346
[242]	validation_0-logloss:0.75456
[243]	validation_0-logloss:0.75493
[244]	validation_0-logloss:0.75510
[245]	validation_0-logloss:0.75525
[246]	validation_0-logloss:0.75527
[247]	validation_0-logloss:0.75505
[248]	validation_0-logloss:0.75559
[249]	validation_0-logloss:0.75560
[250]	validation_0-logloss:0.75667
[251]	validation_0-logloss:0.75721
[252]	validation_0-logloss:0.75749
[253]	validation_0-logloss:0.75830
[254]	validation_0-logloss:0.75898
[255]	validation_0-logloss:0.75864
[256]	validation_0-logloss:0.75978
[257]	validation_0-logloss:0.76078
[258]	validation_0-l

/Users/juanguillenalbarracin/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'CooperativeCreditRisk-XGBoost' already exists. Creating a new version of this model...
Created version '2' of model 'CooperativeCreditRisk-XGBoost'.
